In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from os import walk
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier

In [ ]:
#relax = pd.read_csv("../Muse Data/novonarelax.csv", names = np.arange(0,220))
basePath = "../Muse Data/"
allData = pd.DataFrame()

for (f) in listdir(basePath):
    if not isfile(join(basePath, f)): continue
    if f.startswith("."): continue
    #print(f)
    singleFileData = pd.read_csv(join(basePath, f), header=None)
    singleFileData = singleFileData.iloc[:,0:110] #cut it down like before
    singleFileData['originalRow'] = list(singleFileData.index)
    #print(singleFileData.shape)
    #print(singleFileData.head())
    singleFileData = pd.melt(singleFileData, var_name="originalColumn", id_vars = "originalRow")
    singleFileData['epoch'] = singleFileData["originalRow"].apply(lambda x: x//4)
    singleFileData['chan'] = singleFileData["originalRow"].apply(lambda x: x % 4 + 1)
    singleFileData['variableID'] = singleFileData["originalColumn"].astype(str) +".originalColumn."  +singleFileData["chan"].astype(str) +".chan"
    singleFileData = singleFileData.pivot(index='epoch', values = 'value', columns='variableID')
    singleFileData['source'] = f
    name = f.replace('.csv','')
    name = name.replace('focus','')
    name = name.replace('relax','')
    singleFileData['user'] = name
    if "relax" in f:
        singleFileData['class'] = "relax"
    elif "focus" in f:
        singleFileData['class'] = "focus"
    else:
        continue
    allData = pd.concat([allData, singleFileData])
allData.shape
print(allData.head())

In [137]:
allData['class'] = allData['class'].astype('category')
taskInstancesTarget = allData['class']
taskInstancesTarget = taskInstancesTarget.cat.codes #switch from categories to numbers

taskInstancesData = allData.drop('class', 1)
groups = taskInstancesData['user'] #loose information about subject, use it for cross validation
taskInstancesData = taskInstancesData.drop('source', 1)
taskInstancesData = taskInstancesData.drop('user', 1)
#taskInstancesData = pd.get_dummies(taskInstancesData) #transform catagorical to dummy vars
taskInstancesData

variableID  0.originalColumn.1.chan  0.originalColumn.2.chan  \
epoch                                                          
0                          3.347379                 3.922594   
1                          3.098009                 3.419794   
2                          3.813447                 0.630652   
3                          4.568050                 3.700144   
4                          4.249591                 2.235759   

variableID  0.originalColumn.3.chan  0.originalColumn.4.chan  \
epoch                                                          
0                          0.576974                 4.030018   
1                          4.943621                 2.093604   
2                          4.498111                 3.138631   
3                          3.112263                 3.405895   
4                          4.125033                 1.713787   

variableID  1.originalColumn.1.chan  1.originalColumn.2.chan  \
epoch                                 

variableID  0.originalColumn.1.chan  0.originalColumn.2.chan  \
epoch                                                          
0                          3.347379                 3.922594   
1                          3.098009                 3.419794   
2                          3.813447                 0.630652   
3                          4.568050                 3.700144   
4                          4.249591                 2.235759   
5                          2.921151                 1.247593   
6                          4.149429                 4.015063   
7                          4.791668                 3.844846   
8                          3.064607                 3.125479   
9                          0.734629                 3.552746   
10                         4.069254                 3.929880   
11                         4.201542                 3.161880   
12                         3.891622                 3.959751   
13                         3.120879                 3.300459   
14                         2.254437                 1.940516   
15                         3.604621                 3.699194   
16                         5.851729                 4.160017   
17                         5.893486                 3.639671   
18                         2.645602                 2.512104   
19                         4.659342                 3.846752   
20                         2.697908                 3.335723   
21                         3.538564                 1.752255   
22                         4.632369                 2.506587   
23                         3.885101                 4.271117   
24                         4.092290                 3.894280   
25                         4.683437                 4.631958   
26                         1.692266                 3.489568   
27                         4.424360                 2.223210   
28                         2.976518                 3.166772   
29                         2.053590                 3.572137   
...                             ...                      ...   
10                         5.607295                 3.567706   
11                         5.526713                 5.089861   
12                         5.805463                 6.002592   
13                         5.127918                 4.589741   
14                         4.814339                 5.131501   
15                         6.007476                 4.926854   
16                         5.306625                 4.687372   
17                         3.786949                 1.357976   
18                         5.059399                 4.487735   
19                         5.455513                 3.527765   
20                         5.430236                 6.473985   
21                         4.372993                 5.151210   
22                         6.372010                 5.857866   
23                         5.999632                 5.757645   
24                         5.809400                 3.966558   
25                         2.813381                 3.973847   
26                         5.279986                 3.884931   
27                         5.619839                 5.913186   
28                         5.111303                 5.108199   
29                         4.758999                 5.057867   
30                         4.931864                 4.548870   
31                         6.360191                 1.470537   
32                         3.662398                 4.492980   
33                         4.475780                 5.527996   
34                         3.995213                 3.402875   
35                         5.920833                 5.209316   
36                         4.953954                 3.953080   
37                         5.186008                 4.751489   
38                         4.859955                 5.027041   
39                         4.414

In [143]:
#should be splitting on 'user' - this mixes users
X_train, X_test, y_train, y_test = train_test_split(taskInstancesData, taskInstancesTarget,train_size=0.75, test_size=0.25)

tpot = TPOTClassifier(generations=10, population_size=30, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Optimization Progress:   0%|          | 0/330 [00:00<?, ?pipeline/s]

Version 0.6.8 of tpot is outdated. Version 0.7.0 was released Wednesday March 22, 2017.


Optimization Progress:  10%|█         | 33/330 [00:34<01:37,  3.05pipeline/s]

Generation 1 - Current best internal CV score: 0.5094101885768553


Optimization Progress:  19%|█▉        | 63/330 [00:50<02:08,  2.07pipeline/s]

...

Optimization Progress:  85%|████████▍ | 279/330 [01:45<00:01, 26.17pipeline/s]

Generation 9 - Current best internal CV score: 0.5155830280830281


Optimization Progress:  91%|█████████ | 300/330 [01:45<00:00, 41.57pipeline/s]

Generation 10 - Current best internal CV score: 0.5155830280830281



Best pipeline: BernoulliNB(input_matrix, 0.83000000000000007, 5.0)
0.415019762846
